### Load Envs 

In [1]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()


### ReAct

In [2]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()

#### ReAct Agent
from agents.react.react_agent import ReActAgent

GPT_VER = '0613'
react_agent = ReActAgent(reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=5)

query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2

react_agent.clear_logs()
# answer = react_agent.runnable_agent_step.invoke(query)
# answer = react_agent.runnable_agent_episode.invoke(query, **{'reference':reference})
answer = react_agent.runnable_agent_trial.invoke(query, **{'num_trials':num_trials, 'reference':reference})

[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Thought 1: I need to find out who the former president of Korea is and who the MLB American League MVP of the last season is. Then, I need to find out their ages. I'll start by finding out who the former president of Korea is.
[INFO] Action 1: YDCSearch(query='former president of Korea')
[INFO] Observation 1: The most recent former president of South Korea is Moon Jae-In, who served from 2017 to 2022.
[INFO] Thought 2: Now I know that the former president of Korea is Moon Jae-In. Next, I need to find out his age.
[INFO] Action 2: YDCSearch(query='Moon Jae-In age')
[INFO] Observation 2: Moon Jae-in was born on January 24, 1953.
[INFO] Thought 3: Now I know that Moon Jae-In was born on January 24, 1953. I can calculate his age using the current year. But before that, I need to find out who the MLB America

### Reflexion ReAct

In [3]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()

#### Reflexion Agent
from agents.react.react_agent_with_reflexion import ReActWithReflexionAgent
from agents.react.react_fewshot_examples import REACT_REFLECTION_HEADER, REACT_REFLECTIONS
from chains.reflexion_chains import BaseReflexionChain

GPT_VER = '0613'   
reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/experimental-reflextion", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=REACT_REFLECTIONS
                    )

reflexion_prompt = "jet-taekyo-lee/reflexion-react"
reflexion_agent = ReActWithReflexionAgent(
                        reflexion_chain = reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=5)


query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2

reflexion_agent.clear_logs()
# answer = reflexion_agent.runnable_agent_step.invoke(query)
# answer = reflexion_agent.runnable_agent_episode.invoke(query, **{'reference':reference})
answer = reflexion_agent.runnable_agent_trial.invoke(query, **{'num_trials':num_trials, 'reference':reference})



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Thought 1: I need to find out who the former president of Korea is and who the MLB American League MVP of the last season is. Then, I need to find out their ages. I'll start by finding out who the former president of Korea is.
[INFO] Action 1: YDCSearch(query='former president of Korea')
[INFO] Observation 1: The most recent former president of South Korea is Moon Jae-In, who served from 2017 to 2022.
[INFO] Thought 2: Now that I know the former president of Korea is Moon Jae-In, I need to find out his age.
[INFO] Action 2: YDCSearch(query='Moon Jae-In age')
[INFO] Observation 2: Moon Jae-in was born on January 24, 1953.
[INFO] Thought 3: Now that I know Moon Jae-In was born on January 24, 1953, I can calculate his age. I'll use the current year to do this.
[INFO] Action 3: GetDatetime(IANA_timezone='As

### OpenAI Regacy agent

In [4]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()

# #### OepnAI Agent
GPT_VER = '0613' 
from agents.openai_legacy.openai_legacy_agent import OpenAILegacyAgent
openai_agent = OpenAILegacyAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = "jet-taekyo-lee/openai-function-calling-agent",
    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
    action_word = 'Invoking',
    horizon = 5)


query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2

openai_agent.clear_logs()
# answer = openai_agent.runnable_agent_step.invoke(query)
# answer = openai_agent.runnable_agent_episode.invoke(query, **{'reference':reference})
answer = openai_agent.runnable_agent_trial.invoke(query, **{'num_trials':num_trials, 'reference':reference})



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: The context provided does not specify which former president of South Korea's age is being asked for. There have been multiple former presidents of South Korea. Please provide more specific information.
[INFO] Invoking 2: Could you please specify which former president of Korea you are referring to? There have been multiple former presidents.
[INFO] Answer: Could you please specify which former president of Korea you are referring to? There have been multiple former presidents.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "The former former president of Korea is older. The sum of their age is 99."
[INFO] Trial 2
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: 

### Reflexion OpenAI Legacy agent

In [5]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()

# # #### Reflexion OepnAI Agent
from agents.openai_legacy.openai_legacy_with_reflexion_agent import OpenAILegacyWithReflexionAgent
from agents.openai_legacy.openai_legacy_fewshot_examples import OPENAI_LEGACY_REFLECTIONS, OPENAI_LEGACY_REFLECTION_HEADER
from chains.reflexion_chains import BaseReflexionChain


GPT_VER = '0613'    
openai_legacy_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_LEGACY_REFLECTIONS
                    )


openai_legacy_reflexion_agent = OpenAILegacyWithReflexionAgent(
                        reflexion_chain = openai_legacy_reflexion_chains,
                        reflexion_header = OPENAI_LEGACY_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/rexlexion-openai-function-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word='Invoking',
                        file_logging=True,
                        horizon = 5)


query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2

openai_legacy_reflexion_agent.clear_logs()
# answer = openai_legacy_reflexion_agent.runnable_agent_step.invoke(query)
# answer = openai_legacy_reflexion_agent.runnable_agent_episode.invoke(query, **{'reference':reference})
answer = openai_legacy_reflexion_agent.runnable_agent_trial.invoke(query, **{'num_trials':num_trials, 'reference':reference})



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: The context provided does not specify which former president of South Korea's age you are asking about. There have been multiple former presidents of South Korea. Please provide more specific information.
[INFO] Invoking 2: Could you please specify which former president of Korea you are referring to? There have been multiple former presidents of Korea.
[INFO] Answer: Could you please specify which former president of Korea you are referring to? There have been multiple former presidents of Korea.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "The former former president of Korea is older. The sum of their age is 99."
[INFO] Trial 2
[INFO] Reflexion......
[INFO] You have attempted to answer following q

### Parallel Function Calling agent

In [1]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
import asyncio
config = HD22Configuration()

#### Parallel Function Calling Agent
GPT_VER = '1106' 
from agents.openai_parallel.openai_parallel_agent import OpenAIParallelAgent

openai_parallel_agent = OpenAIParallelAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
    action_word = 'Tool invocations in parallel',
    observation_word = 'Tool-invocations results in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
    horizon = 5
)



query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2


openai_parallel_agent.clear_logs()

# await openai_parallel_agent.runnable_agent_step.ainvoke(query)
# await openai_parallel_agent.runnable_agent_episode.ainvoke(query, **{'reference':reference})
await openai_parallel_agent.runnable_agent_trial.ainvoke(query, **{'reference':reference, 'num_trials':num_trials})



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of Korea age), Tool 2-> YDCSearch(query=last season MLB American League MVP age)]
[INFO] (step 1-observation) :: [Tool 1-> The age of a former president of South Korea would depend on which former president you are referring to. As of my knowledge cutoff in 2023, here are the ages of the living former presidents:

1. Lee Myung-bak (이명박) - Born December 19, 1941, so he would be 81 years old.
2. Park Geun-hye (박근혜) - Born February 2, 1952, so she would be 71 years old.

Please note that these ages are based on the birthdates provided and the current year being 2023. If you are asking about a specific former president, please provide their name for a more accurate response., Tool 2-> The American League MVP for the 202

RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 1-observation) :: [Tool 1-> The former president of South Korea is Park Geun-hye. She served as the 11th president of South Korea from 2013 until her impeachment in 2017. Park was the first woman to be elected as president of South Korea and the first female president popularly elected as head of state in East Asia. Her presidency ended when she was impeached and subsequently convicted on charges related to corruption. Park Geun-hye was pardoned in December 2021 by then-President Moon Jae-in., Tool 2-> The American League MVP for the last season (2022) was Aaron Judge of the New York Yankees.]
[INFO] (step 2-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=Park Geun-hye birthdate), Tool 2-> YDCSearch(query=Aaron Judge birthdate)]
[INFO] (step 2-observation) :: [Tool 1-> Park Geun-hye was born on February 2, 1952., Tool 2-> Aaron Judge's birthday is on April 26.]
[INFO] (step 3-action) Tool invocations in parallel: [Tool 1-> PythonREPL(command=print(2023 - 19

### Reflexion + Parallel Function Calling agent

In [2]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio
config = HD22Configuration()

#### Reflexion + Parallel Function Calling Agent
from chains.reflexion_chains import BaseReflexionChain
from agents.openai_parallel.openai_parallel_with_reflexion_agent import OpenAIParallelWithReflexionAgent
from agents.openai_parallel.openai_parallel_fewshot_examples import OPENAI_PARALLEL_REFLECTIONS, OPENAI_PARALLEL_REFLECTION_HEADER

GPT_VER = '1106'    
openai_parallel_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-parallel-calling-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_PARALLEL_REFLECTIONS
                    )


openai_parallel_reflexion_agent = OpenAIParallelWithReflexionAgent(
                        reflexion_chain = openai_parallel_reflexion_chains,
                        reflexion_header = OPENAI_PARALLEL_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/reflexion-openai-parallel-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word = 'Tool invocations in parallel',
                        observation_word = 'Tool-invocations results in parallel',
                        use_chat_completion_api = True,
                        azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
                        file_logging=True,
                        horizon = 5)

query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2


openai_parallel_reflexion_agent.clear_logs()
# await openai_parallel_reflexion_agent.runnable_agent_step.ainvoke(query)
# await openai_parallel_reflexion_agent.runnable_agent_episode.ainvoke(query, **{'reference':reference})
await openai_parallel_reflexion_agent.runnable_agent_trial.ainvoke(query, **{'reference':reference, 'num_trials':num_trials})



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of South Korea), Tool 2-> YDCSearch(query=MLB American League MVP last season)]
[INFO] (step 1-observation) :: [Tool 1-> Park Geun-hye is a former president of South Korea who served from 2013 until her impeachment in 2017. She was the country's first female president and the first president to be born after the founding of South Korea. Her presidency ended when she was impeached and removed from office due to a corruption scandal involving her close confidante, Choi Soon-sil. Park was subsequently convicted on related charges and sentenced to prison. She was pardoned and released from prison in December 2021., Tool 2-> The American League MVP for the last season (2023) was Shohei Ohtani of the LA Angels.]
[INFO] (s

RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 2-observation) :: [Tool 1-> Park Geun-hye was born on February 2, 1952., Tool 2-> Shohei Ohtani was born on July 5, 1994.]
[INFO] (step 3-action) Tool invocations in parallel: [Tool 1-> PythonREPL(command=print(2023 - 1952 + 2023 - 1994))]
[INFO] (step 3-observation) :: [Tool 1-> 100]
[INFO] (step 4-action) Tool invocations in parallel: Now I can answer directly without resorting to any tool.
[INFO] Answer: Park Geun-hye is older than Shohei Ohtani. If you sum up their ages, the result is 100.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "The former former president of Korea is older. The sum of their age is 99."
[INFO] Trial 2
[INFO] Reflexion......
[INFO] You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
- Reflection: The error in the previous 

RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 1-observation) :: [Tool 1-> Former President Park Geun-hye of South Korea served as the 11th president from 2013 until her impeachment in 2017. She was the first woman to be elected as president in South Korea and the first female president popularly elected as head of state in East Asia. Park was impeached and removed from office on charges of corruption and abuse of power, and she was subsequently convicted and imprisoned. However, she was pardoned by her successor, President Moon Jae-in, in December 2021, citing her deteriorating health and the need to promote national unity. Park was released from prison and returned home in March 2022., Tool 2-> The American League MVP for the last season (2023) was Shohei Ohtani of the LA Angels.]
[INFO] (step 2-action) Tool invocations in parallel: [Tool 1-> GetDatetime(IANA_timezone=Asia/Seoul), Tool 2-> YDCSearch(query=Park Geun-hye date of birth), Tool 3-> YDCSearch(query=Shohei Ohtani date of birth)]
[INFO] (step 2-observation) 